* Difference between the two exponentially weighted moving averages with long position only. (8/8)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
eth_b = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Back/ETH200101-230815.csv')
eth_f = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Walk_forward/ETH230816-240829.csv')
#low sharpe

In [ ]:
sol_b = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Back/SOL200101-230815.csv')
sol_f = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Walk_forward/SOL230816-240829.csv')
# not beat bnh

In [ ]:
btc_b = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Back/BTC200101-230815.csv')
btc_f = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Walk_forward/BTC230816-240829.csv')
#low sharpe

In [ ]:
bnb_b = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Back/BNB200101-230815.csv')
bnb_f = pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Walk_forward/BNB230816-240829.csv')
#low sharpe

In [ ]:
# backtesting

def calculate_ema(prices, days, smoothing=2):
  ema = [sum(prices[:days]) / days]  #ema = [ ] , meaning have index for each days and later can append
  for price in prices[days:]:
    ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
  return ema

def backtesting(df: pd.DataFrame, long_window: int, short_window: int, plot: bool = False) -> Optional[pd.Series]:

    df = df[['Date', 'Close']].copy()  #to avoid warning message
    df['pct_change']=df['Close'].pct_change()

    long_ema = calculate_ema(df['Close'], long_window)
    short_ema = calculate_ema(df['Close'], short_window)

    df['long_ema'] = [np.nan] * (long_window - 1) + long_ema

    df['short_ema'] = [np.nan] * (short_window - 1) + short_ema

    df['pos'] = np.where(df['short_ema'] > df['long_ema'], 1, np.where(df['short_ema'] < df['long_ema'], -1,0))

    df['pos_t-1'] = df['pos'].shift(1)
    df['trade'] = abs(df['pos_t-1'] - df['pos'])
    df['pnl'] = df['pos_t-1'] * df['pct_change'] - df['trade']*0.05/100
    df['cumu'] = df['pnl'].cumsum()
    df['dd'] = df['cumu'].cummax() - df['cumu']
    df['bnh_cumu']= df['pct_change'].cumsum()

    annual_return = round(df['pnl'].mean() * 365,2)

    # avoid division of zero
    if df['pnl'].std() != 0:
        sharpe = round((df['pnl'].mean() / df['pnl'].std()) * np.sqrt(365), 2)
    else:
        sharpe = np.nan

    mdd = df['dd'].max()

    # avoid division of zero
    if mdd != 0:
        calmar = round(annual_return / mdd,2)
    else:
        calmar = np.nan

    if plot:
      fig = px.line(df,x='Date', y=['cumu','bnh_cumu'])
      fig.show()
      return

    return pd.Series([long_window,short_window, sharpe, calmar, annual_return, mdd],index= ['long_window','short_window','sharpe', 'calmar','annualR','mdd'])

In [ ]:
# optimisation

long_window_list = np.arange(20, 80, 2)
short_window_list = np.arange(2, 20, 2)
result_list=[]

for long_ema in long_window_list:
    for short_ema in short_window_list:
        result_list.append(backtesting(df = bnb_b, long_window = long_ema, short_window = short_ema))

result_df = pd.DataFrame(result_list)
result_df= result_df.sort_values(by = 'sharpe',ascending=False)
result_df.head(10)

In [ ]:
# plot cumu return against bnh
backtesting(df =sol_f, long_window = 20, short_window = 16, plot = True)